# Recognizing Emotions from Facial Expression Data

The data comes from a Kaggle competition:

https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Assuming the data has been downloaded to a CSV file
df = pd.read_csv('fer2013.csv')
df

In [ ]:
emotion_labels=['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
# Put data into Numpy arrays
X = -1 * np.ones((len(df), 48, 48))
for i in range(len(df)):
    X[i,:,:] = np.array([int(s) for s in df['pixels'].loc[i].split()]).reshape(48,48)

In [ ]:
y = df['emotion'].values

In [ ]:
# display a random picture from the data set with label
row_index = np.random.choice(len(df))
plt.imshow(X[row_index, :, :], cmap='gray')
plt.title(emotion_labels[y[row_index]])